In [1]:
import numpy as np
import pandas as pd
import datetime as dt

### 22.12.04 记录：

### 数据说明：

- 最终panel(包含`duration`): `park_panel.csv`, `park_panel.parquet`

- 不含`duration`的panel: `1204_panel.csv`,`1204_panel.parquet`

- 空panel(运行`duration2`函数用):`void_panel.csv` ：直接读取出来的不能用，需要先转换为tibble格式那种，代码见2.

- 最新的原始数据：`2021_q2_new.csv`

- 最新的有效车位id： `1202_valid.csv`

- 最新的处理过后的数据集：`1204_res.csv`,`1204_res.parquet`

--------------------------------------------------------

### 主要处理：

- 构建panel.

- 时间分隔函数: @function: `duration2()`.
  
要在panel里赋值！

--------------------------------------------------------

### MENU:

- 1.读取`res`.

- 2.读取void_panel.

- 3.时间间隔函数（update）

--------------------------------------------------------

1. 读取`res`:

In [2]:
path = r'C:\Users\yyp\Desktop\Upenn\MUSA\courses\MUSA5080\Final\data\1204_res.csv'
res=pd.read_csv(path)

- 运用`duration2`函数需要将`res`中的开始、结束、向下取整的小时转化为时间格式。

In [3]:
res["session_start_dt"] = pd.to_datetime(res["session_start_dt"])
res["session_end_dt"] = pd.to_datetime(res["session_end_dt"])
res["interval60"]=pd.to_datetime(res["interval60"])

In [4]:
res

,Unnamed: 0,post_id,session_start_dt,session_end_dt,interval,interval60
0,0,102-02990,2021-04-01 09:39:52,2021-04-01 11:39:52,7200,2021-04-01 09:00:00
1,1,102-02990,2021-04-01 12:11:01,2021-04-01 12:48:31,2250,2021-04-01 12:00:00
2,2,102-02990,2021-04-01 13:26:19,2021-04-01 13:56:19,1800,2021-04-01 13:00:00
3,3,102-02990,2021-04-01 14:11:03,2021-04-01 15:06:19,3316,2021-04-01 14:00:00
4,4,102-02990,2021-04-01 15:49:00,2021-04-01 16:09:00,1200,2021-04-01 15:00:00
...,...,...,...,...,...,...
2878229,2878229,920-00350,2021-06-25 14:07:06,2021-06-25 14:10:48,222,2021-06-25 14:00:00
2878230,2878230,920-00350,2021-06-25 14:10:53,2021-06-25 14:15:30,277,2021-06-25 14:00:00
2878231,2878231,920-00350,2021-06-29 13:04:54,2021-06-29 15:37:13,9139,2021-06-29 13:00:00
2878232,2878232,920-00350,2021-06-29 15:43:02,2021-06-29 16:38:26,3324,2021-06-29 15:00:00


---------------------------------

2.读取void_panel。

- 这个是`duration2`函数里面的`void_panel`

In [15]:
path = r'C:\Users\yyp\Desktop\Upenn\MUSA\courses\MUSA5080\Final\data\void_panel.csv'
void_panel=pd.read_csv(path)

- 转成panel形式，计算速度非常快。

In [17]:
void_panel = void_panel.groupby(["post_id","interval60"]).agg("first")

In [18]:
void_panel

duration
post_id   interval60                   
102-02990 2021-04-01 09:00:00         0
          2021-04-01 10:00:00         0
          2021-04-01 11:00:00         0
          2021-04-01 12:00:00         0
          2021-04-01 13:00:00         0
...                                 ...
920-00350 2021-06-30 13:00:00         0
          2021-06-30 14:00:00         0
          2021-06-30 15:00:00         0
          2021-06-30 16:00:00         0
          2021-06-30 17:00:00         0

[13017186 rows x 1 columns]

--------------------------------------

3.时间间隔函数（update）

- `duration2`: 在panel中的计算。

In [ ]:
#for rowNumber in range(2878234):
# void_panel是一个空panel.

def duration2(rowNumber):
    start_time_rounded = res.loc[rowNumber, "interval60"]
    start_time = res.loc[rowNumber, "session_start_dt"]
    end_time = res.loc[rowNumber, "session_end_dt"]
    post_id_row = res.loc[rowNumber, "post_id"]
    
    while start_time<end_time:
        s = start_time + onehour - (start_time.minute*onemin+start_time.second*onesec)
        
        if(end_time>s):
            this_end=s
        else:
            this_end=end_time
            
        this_interval = (this_end-start_time).total_seconds()
        p0 = void_panel.loc[(post_id_row,start_time_rounded),"duration"]
        void_panel.loc[(post_id_row,start_time_rounded),"duration"] = p0 + this_interval
        start_time_rounded += onehour
        start_time = this_end
    
    if rowNumber%1000==0:
    print(rowNumber)
   

In [ ]:
for i in range(2878234):
    duration2(i)

- `park_panel` 为运行以上函数得到的结果。

In [5]:
path = r'C:\Users\yyp\Desktop\Upenn\MUSA\courses\MUSA5080\Final\data\park_panel.csv'
park_panel=pd.read_csv(path)

In [9]:
park_panel

,post_id,interval60,duration
0,102-02990,2021-04-01 09:00:00,1208
1,102-02990,2021-04-01 10:00:00,3600
2,102-02990,2021-04-01 11:00:00,2392
3,102-02990,2021-04-01 12:00:00,2250
4,102-02990,2021-04-01 13:00:00,1800
...,...,...,...
13017181,920-00350,2021-06-30 13:00:00,0
13017182,920-00350,2021-06-30 14:00:00,887
13017183,920-00350,2021-06-30 15:00:00,2770
13017184,920-00350,2021-06-30 16:00:00,0


- tibble 形式的park_panel,按以下处理即可。

In [10]:
park_panel.groupby(["post_id","interval60"]).agg("first")

duration
post_id   interval60                   
102-02990 2021-04-01 09:00:00      1208
          2021-04-01 10:00:00      3600
          2021-04-01 11:00:00      2392
          2021-04-01 12:00:00      2250
          2021-04-01 13:00:00      1800
...                                 ...
920-00350 2021-06-30 13:00:00         0
          2021-06-30 14:00:00       887
          2021-06-30 15:00:00      2770
          2021-06-30 16:00:00         0
          2021-06-30 17:00:00         0

[13017186 rows x 1 columns]